In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.neighbors import NearestNeighbors
import pickle

# ---------------------------
# Nearest Neighbors Approach Setup
# ---------------------------

# Load the dataset
equipments_df = pd.read_csv('Equipments.csv')

# Define features (for the first part, using the original approach)
categorical_features = ['Type', 'Material', 'Special Features']
numerical_features = ['Weight (kg)', 'Durability (years)', 'Protection Level', 'Price (INR)']

# Load the dataset again for Nearest Neighbors approach
file_path = 'Equipments.csv'  # Replace with your dataset path
equipments_data = pd.read_csv(file_path)

# Filter data for relevant columns
relevant_columns = ["Type", "Material", "Weight (kg)", "Durability (years)", "Protection Level", "Special Features", "Price (INR)"]
filtered_data = equipments_data[relevant_columns]

# For this part, redefine categorical features with 4 items
categorical_features_nn = ["Type", "Material", "Protection Level", "Special Features"]
numerical_features_nn = ["Weight (kg)", "Durability (years)", "Price (INR)"]

# One-hot encoding and standardizing
encoded_data = pd.get_dummies(filtered_data, columns=categorical_features_nn)
scaler_nn = StandardScaler()
encoded_data[numerical_features_nn] = scaler_nn.fit_transform(encoded_data[numerical_features_nn])

# Build the recommendation model (Nearest Neighbors)
recommendation_model = NearestNeighbors(n_neighbors=5, metric='euclidean')
recommendation_model.fit(encoded_data)

# Function to recommend top 5 equipment using Nearest Neighbors
def recommend_equipment(input_features):
    # Preprocess input features to match the dataset format
    input_df = pd.DataFrame([input_features])
    input_encoded = pd.get_dummies(input_df, columns=categorical_features_nn)
    input_encoded = input_encoded.reindex(columns=encoded_data.columns, fill_value=0)
    input_encoded[numerical_features_nn] = scaler_nn.transform(input_encoded[numerical_features_nn])
    
    # Find nearest neighbors (this returns exactly 5 neighbors)
    distances, indices = recommendation_model.kneighbors(input_encoded)
    recommended_items = equipments_data.iloc[indices[0]]
    
    # Ensure only 5 recommendations are used (in case of any unexpected extra rows)
    recommended_items = recommended_items.head(5)
    
    # Display recommendations with sequential numbering (1 to 5)
    for rank, (_, row) in enumerate(recommended_items.iterrows(), start=1):
        print(f"\nEquipment {rank}:")
        for col in equipments_data.columns:
            print(f"• {col}: {row[col]}")
    return recommended_items

# Example Usage for Nearest Neighbors Approach
input_features = {
    "Type": "Body Armor",  
    "Material": "Kevlar",  
    "Weight (kg)": 2.5,
    "Durability (years)": 5,
    "Protection Level": "NIJ Level IIIA",
    "Special Features": "Lightweight and Flexible",
    "Price (INR)": 15000
}

recommendations = recommend_equipment(input_features)





Equipment 1:
• Item ID: 1
• Name: Kevlar Vest
• Type: Body Armor
• Manufacturer/Brand: ArmorTech Industries
• Material: Kevlar
• Weight (kg): 2.5
• Durability (years): 5
• Dimensions: nan
• Protection Level: NIJ Level IIIA
• Special Features: Lightweight and Flexible
• Introduced Year: 2010
• Price (INR): 15000
• Usage: Personal Protection
• Popularity: High
• Legal Restrictions: Restricted

Equipment 2:
• Item ID: 3
• Name: Bulletproof Jacket
• Type: Body Armor
• Manufacturer/Brand: SafeGuard Armor
• Material: Ceramic and Kevlar
• Weight (kg): 3.0
• Durability (years): 5
• Dimensions: nan
• Protection Level: NIJ Level III
• Special Features: Multilayer Design
• Introduced Year: 2015
• Price (INR): 20000
• Usage: Personal Protection
• Popularity: High
• Legal Restrictions: Restricted

Equipment 3:
• Item ID: 9
• Name: Flak Jacket
• Type: Body Armor
• Manufacturer/Brand: ArmorPro Solutions
• Material: Nylon and Steel
• Weight (kg): 4.0
• Durability (years): 5
• Dimensions: nan
• Protec

In [22]:
# Store all components in a dictionary
model = {
    'df': equipments_df,  # Full dataset
    'encoder': encoder,
    'scaler': scaler,
    'processed_df': processed_df,
    'categorical_features': categorical_features,
    'numerical_features': numerical_features
}

# Pickle the model
with open('equipment_recommendation_model.pkl', 'wb') as file:
    pickle.dump(model, file)

print("Model has been pickled successfully!")

Model has been pickled successfully!
